In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from dateutil import parser
from datetime import datetime
from prepare_data import read_data

In [2]:
incidents = read_data()

In [3]:
incidents.head()

,Unnamed: 0,RESPONSE TIME,MINS OF DAY,DAY OF WEEK,DAY OF MONTH,DAY OF YEAR,MONTH OCCURRED,YEAR OCCURRED,LATITUDE,LONGITUDE,CRIME CATEGORY ARRAY,GEO CODE ARRAY
0,1,14.0,1138.0,1.0,18.0,169.0,6,2019,NaN,NaN,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,7.0,945.0,4.0,28.0,241.0,8,2020,40.116511,-88.210833,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7,9.0,665.0,4.0,8.0,312.0,11,2013,40.101881,-88.200156,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,9,6.0,1176.0,0.0,23.0,328.0,11,2020,NaN,NaN,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,11,6.0,1044.0,6.0,22.0,327.0,11,2020,NaN,NaN,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
